# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**You must implement your own work.** You can and are encouraged to use helper libraries like `numpy`, but you must not use 3rd-party libraries or code that solves the assignment for you. *Even if it is properly referenced*, you only get marks for work you have done. If you are unsure, ask on the forum or ask a tutor directly.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

You should also consider what *data structures* you use within your code. Certain algorithms will work much faster with certain data structures, and understanding this is considered part of understanding the algorithm itself. 

I suggest you implement *constraint satisfaction* as it is described in the unit material. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%). There is an example of constraint satisfaction on the eight-queens puzzle, and you could use this as a guide to help you get started.

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [ ]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested.


In [ ]:
import copy

import numpy as np


class SudokuHelpUtils:
    """Helpful utilities used to organise sudoku"""

    def __init__(self):
        """Create the helper lists and populate them"""
        self.squares_list = []
        self.__get_sudoku_elements()
        self.row_list = []
        self.__get_row_relations()
        self.column_list = []
        self.__get_column_relations()
        self.box_list = []
        self.__get_box_relations()
        self.unit_list = []
        self.peer_list = []
        self.__generate_peers()

    def __cross_product(self, A, B) -> list:
        """Creates the cross product of two given lists."""

        return [(a, b) for a in A for b in B]

    def __get_sudoku_elements(self):
        """Gets a list containing the coordinates of every sudoku element."""
        rows = list(range(9))
        columns = list(range(9))
        self.squares_list = self.__cross_product(rows, columns)

    def __get_row_relations(self):
        """Creates a list containing the coordinates of all the sudoku's rows"""

        for i in range(9):
            self.row_list.append([(row, i) for row in range(9)])

    def __get_column_relations(self):
        """Creates a list containing the coordinates of all the sudoku's columns"""

        for i in range(9):
            self.column_list.append([(i, column) for column in range(9)])

    def __get_box_relations(self):
        """Creates a list containing the coordinates of all the boxes in a 9x9 sudoku"""

        # Iterate through all the 3xe boxes of the sudoku.
        for i in [[0, 1, 2], [3, 4, 5], [6, 7, 8]]:
            for j in [[0, 1, 2], [3, 4, 5], [6, 7, 8]]:
                self.box_list.append(self.__cross_product(i, j))

    def get_units(self) -> list:
        """Gets each coordinate's related elements from rows, columns and its box, including the coordinate itself."""

        # Concatenate all the list relations.
        all_relations = (self.row_list + self.column_list + self.box_list)

        # Go through every coordinate and get lists related to that coordinates
        # in rows, columns, and the squares.
        all_unit_relations = [[u for u in all_relations if s in u] for s in self.squares_list]
        # Split the list into nested lists of 9 elements each.
        self.unit_list = np.array_split(all_unit_relations, 9)
        return self.unit_list

    def __generate_peers(self):
        """Get each coordinates related elements from rows, columns and its box, excluding the coordinate itself in a
           "flat-ish" list"""

        # for s in self.squares_list:
        for s in self.squares_list:
            peers = np.array(self.get_units()[s[0]][s[1]])
            # Flatten last layer of the nested list.
            peers = (peers.reshape(-1, peers.shape[-1]))
            # Remove repeated coordinates
            peers = np.unique(peers, axis=0)
            peers = [x for x in peers if not np.array_equal(x, s)]
            self.peer_list.append(peers)
        # print(self.peer_list)
        self.peer_list = np.array_split(self.peer_list, 9)

    def get_peers(self):
        """Gets the peers"""
        return self.peer_list

    def to_backtrack_or_not(self, values):
        """If the search successfully returns values, continue,
           otherwise return false"""
        for items in values:
            if items != False:
                return items
        # If the assert function returns false due to breaking a constraint
        # then some function returns false
        return False


def init_sudoku(sudoku):
    """Initialises the sudoku values grid and reads the given sudoku into the grid."""

    # Create grid 9x9 grid of lists containing the integers 1-9.
    values = [[list(range(1, 10)) for _ in range(9)] for _ in range(9)]
    for row in range(9):
        for column in range(9):
            square_value = sudoku[row][column]
            # print(row, column, square_value)
            if square_value in [1, 2, 3, 4, 5, 6, 7, 8, 9] and not assign_value(values, (row, column), square_value):
                return False  # We cant assign d to square square_value
    return values


def assign_value(values: list, coordinates: tuple, square_value: int):
    """Assign a value to an element."""

    other_values = values[coordinates[0]][coordinates[1]].copy()

    # If value does not exist already in the list, produce error
    # This prevents double related numbers in the sudoku grid.
    try:
        other_values.remove(square_value)
    except ValueError:
        return False
    # print(other_values)
    # print(values[coordinates[0]][coordinates[1]])

    # Remove the value from the element and its related elements.
    if all(remove_related_values(values, coordinates, possible_values) for possible_values in other_values):
        return values
    else:
        return False


def remove_related_values(values: list, coordinates: tuple, square_value: int):
    """Remove related values from the element's coordinates.s"""

    # print(values)
    if square_value not in values[coordinates[0]][coordinates[1]]:
        return values
    # print(values[coordinates[0]][coordinates[1]])
    values[coordinates[0]][coordinates[1]].remove(square_value)
    # print(values[coordinates[0]][coordinates[1]])

    # If an element eventually reaches one possible square value then remove it from its related elements.
    if len(values[coordinates[0]][coordinates[1]]) == 0:
        return False
    elif len(values[coordinates[0]][coordinates[1]]) == 1:
        other_values = values[coordinates[0]][coordinates[1]][0]
        if not all(remove_related_values(values, peer_coordinates, other_values) for peer_coordinates in
                   sudoku_help_utils.get_peers()[coordinates[0]][coordinates[1]]):
            return False

    # If a unit is reduced to only one place then put it there.
    for unit in sudoku_help_utils.get_units()[coordinates[0]][coordinates[1]]:
        unit_values = [coordinates for coordinates in unit if square_value in values[coordinates[0]][coordinates[1]]]
        if len(unit_values) == 0:
            return False
        elif len(unit_values) == 1:
            if not assign_value(values, unit_values[0], square_value):
                return False
    return values


def backtrack_search(values: list):
    """Perform a backtracking search."""

    if values is False:
        return False

    # Terminating condition for checking if the sudoku is solved.
    if all(len(values[coordinate[0]][coordinate[1]]) == 1 for coordinate in sudoku_help_utils.squares_list):
        return values

    # Pick the element with the smallest amount of values
    next_coordinate = min(
        coordinate for coordinate in sudoku_help_utils.squares_list if len(values[coordinate[0]][coordinate[1]]) > 1)

    # Try to assign the possible values to the element, using a copy of the values.
    return sudoku_help_utils.to_backtrack_or_not(
        backtrack_search(assign_value(copy.deepcopy(values), next_coordinate, next_value)) for next_value in
        values[next_coordinate[0]][next_coordinate[1]])


def sudoku_solver(sudoku: list):
    """Driver code for sudoku"""

    global sudoku_help_utils
    sudoku_help_utils = SudokuHelpUtils()

    # Nested functions clear my very messy global namespace :))
    # Don't complain.
    def return_sudoku(values: list):
        """Return sudoku when solved, or if search failed, return the failure sudoku."""

        # If search fails return 9x9 numpy array of -1s.
        if values == False:
            return np.array([[-1] * 9] * 9)

        formatted_values = [[] for _ in range(9)]
        for i in range(9):
            for j in range(9):
                formatted_values[i].append(values[i][j][0])
        # print(formatted_values)
        return formatted_values

    def solve(sudoku_grid: list):
        return backtrack_search(init_sudoku(sudoku_grid))

    return return_sudoku(solve(sudoku))


All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up – many more than 15 of each. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 20 seconds each on the test machine to count as successful – if it takes longer, it will timeout. Note that this is a maximum, not a goal. Higher grades require better performance on harder puzzles. As a very rough benchmark, you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms and data structures. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [ ]:
SKIP_TESTS = True

import numpy as np

def tests():
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break
            
if not SKIP_TESTS:
    tests()

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [ ]:
def submission_tests():
    import sys
    import pathlib

    fail = False;

    if not SKIP_TESTS:
        fail = True;
        print("You must set the SKIP_TESTS constant to True in the cell above.")

    p1 = pathlib.Path('./readme.txt')
    p2 = pathlib.Path('./readme.md')
    if not (p1.is_file() or p2.is_file()):
        fail = True;
        print("You must include a separate file called readme.txt or readme.md in your submission.")

    p3 = pathlib.Path('./sudoku.ipynb')
    if not p3.is_file():
        fail = True
        print("This notebook file must be named sudoku.ipynb")

    if "sudoku_solver" not in globals():
        fail = True;
        print("You must include a function called sudoku_solver which accepts a numpy array.")
    else: 
        sudoku = np.load("data/very_easy_puzzle.npy")[0]
        solution = np.load("data/very_easy_solution.npy")[0]

        if not np.array_equal(sudoku_solver(sudoku), solution):
            print("Warning:")
            print("Your sudoku_solver function does not correctly solve the first sudoku.")
            print()
            print("Your assignment is unlikely to get any marks from the autograder. While we will")
            print("try to check it manually to assign some partial credit, we encourage you to ask")
            print("for help on the forum or directly to a tutor.")
            print()
            print("Please use the readme file to explain your code anyway.")

    if fail:
        print()
        sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
    else:
        print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
        print("assignment page, without changing any filenames.")
        print()
        print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")
        
submission_tests()

In [ ]:
# This is a TEST CELL. Do not delete or change.